# Reprojection and Regridding

This notebook demonstrates regridding input data for the pan-Arctic datasets

In [ ]:
from pathlib import Path

import rioxarray as riox
import xarray as xr

from affine import Affine

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from pyproj import CRS

DATAPATH = Path(".") / ".." / "data"

example_file = DATAPATH / "NSIDC0051_SEAICE_PS_N25km_20220531_v2.0.nc"

`rioxarray.rio.project` needs information about the target projection and grid definition.  Projection information can be provided as an EPSG code, a proj4 string or a Well Known Text (WKT).  Unfortunately, the EPSG code for the EASE Grid North Projection (3408) has been depreciated.  `gdal` defaults to the EASE Grid v2.0 North projection.  **This is the wrong projection**. So the projection must be defined using either a proj4 string or WKT.  We use a proj4 string. 

This is because of the reference datum not being fully specified _confirm this_.

In [ ]:
EASE_Grid_North = "+proj=laea +lat_0=90 +lon_0=0 +x_0=0 +y_0=0 +R=6371228 +units=m +no_defs +type=crs"

In [ ]:
EASEGridNorth = CRS.from_proj4(EASE_Grid_North)

In [ ]:
EASEGridNorth.to_dict()

The grid definition tells the reprojection tools the size of grid cells, the size of the grid and the x and y projected coordinates of the grid.  We want to reproject and resample the data to the 721 x 721 25 km EASE Grid.  The grid parameters for this grid are:

| Parameter | Value |
|-----------|-------|
| Number of Columns | 721 |
| Number of Rows | 721 |
| Pixel width in m | 25067.53 | 
| Pixel height in m | -25067.53 | 
| x-axis map coordinate of the outer edge of the upper-left pixel | -9036842.76	|
| y-axis map coordinate of the outer edge of the upper-left pixel |  9036842.76 |

This information is taken from this table of [EASE Grid parameters](https://nsidc.org/data/user-resources/help-center/guide-ease-grids#anchor-9).  You'll notice that in the table above, the pixel height is negative.  This is because the grid origin is the corner of the upper left pixel.  Pixels below this upper-left pixel will be less than the y-axis coordinate _Not the best description_.

In [ ]:
columns = 721
rows = 721
pixel_width_m = 25067.5
pixel_height_m = -25067.5
grid_origin_x_m = -9036842.76
grid_origin_y_m = 9036842.76

We can now define a `GeoTransform`.  This is an Affine matrix that is used by gdal to calculate x and y coordinates of pixels.

$$
$$

In [ ]:
geotransform = (grid_origin_x_m, pixel_width_m, 0., grid_origin_y_m, 0., pixel_height_m)
grid_definition = Affine.from_gdal(*geotransform)
grid_definition

In [ ]:
ds = xr.open_dataset(example_file, decode_coords="all")
ds

In [ ]:
ds_ease = ds.rio.reproject(EASE_Grid_North, transform=grid_definition, shape=(columns, rows))
ds_ease

In [ ]:
EASEGridNorth_ccrs = ccrs.CRS(EASEGridNorth.to_dict())

In [ ]:
EASEGridNorth_ccrs

In [ ]:
EASEGridNorth_ccrs = ccrs.LambertAzimuthalEqualArea(central_longitude=0., central_latitude=90., 
                                                    globe=ccrs.Globe(semimajor_axis=6371228))
fig = plt.figure(figsize=(12, 7))

ax1 = fig.add_subplot(121, projection=EASEGridNorth_ccrs)
ax2 = fig.add_subplot(122, projection=EASEGridNorth_ccrs)

ds.F17_ICECON[0,:,:].plot.imshow(ax=ax1, interpolation="none")
#ax1.set_extent()
    
ds_ease.F17_ICECON[0,:,:].plot.imshow(ax=ax2, interpolation="none")

In [ ]:
APPX_PATH = Path("/home/apbarret/Data/Sunlight_under_seaice/POLAR_APP_X") / "1982"

In [ ]:
appx_filelist = list(APPX_PATH.glob("*"))

In [ ]:
appxf = appx_filelist[0]

In [ ]:
variables = [
    'cdr_surface_albedo',
    'cdr_surface_downwelling_shortwave_flux',
    'cdr_surface_temperature',
]
ds = xr.open_dataset(appxf, )
ds